In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("./src/")
sys.path.append("../dift/src/models")

In [4]:
from utils import load_frames
frames, frames_fpaths = load_frames('../test_videos/frames/bmx_short/color/')
print(f'loaded {len(frames)} frames')

loaded 158 frames


In [5]:
from src.keypoint_extractor import KeypointExtractor
extractor = KeypointExtractor(device='cuda:1')

# Run extractor functions separately

In [ ]:
frames = frames[:10]

In [ ]:
features = extractor.img_to_features(frames, 'bmx biker in a skate park')

In [ ]:
src_ft = features[0].unsqueeze(0)
target_features = features[6:7]
correspondences = extractor.get_correspondence(src_ft, target_features, (100, 100))

In [ ]:
correspondences

# Video Tracking

In [6]:
from performance import PerformanceManager

In [15]:
perf_manager = PerformanceManager()
extractor.track_keypoints(frames[:10], prompt='biker in a skate park', source_frame_idx=0, grid_size=10, perf_manager=perf_manager)

Tracking keypoints:  20%|██        | 2/10 [00:29<01:58, 14.78s/frame]

In [13]:
perf_manager.print_summary()

+-----------------------------------+------------------+--------------------------+--------------+----------------+
|                Name               | Average Time (s) | 95th Percentile Time (s) | Num recorded | Total time (s) |
+-----------------------------------+------------------+--------------------------+--------------+----------------+
|    extract_features_from_image    |      0.3995      |          0.4163          |      10      |     3.9954     |
|         upsample_features         |      0.8434      |          0.8434          |      1       |     0.8434     |
|        init_cos_similarity        |      0.0002      |          0.0002          |     900      |     0.1422     |
|           cos_similarity          |      0.0143      |          0.0145          |     900      |    12.9023     |
|      get_max_xy_from_cos_map      |      0.0001      |          0.0002          |     900      |     0.1158     |
| get_frame_to_frame_correspondence |     25.7766      |         29.2376

+-----------------------------------+------------------+--------------------------+--------------+----------------+
|                Name               | Average Time (s) | 95th Percentile Time (s) | Num recorded | Total time (s) |
+-----------------------------------+------------------+--------------------------+--------------+----------------+
|    extract_features_from_image    |      0.3611      |          0.4035          |      2       |     0.7222     |
|         upsample_features         |      0.1753      |          0.1753          |      1       |     0.1753     |
|        init_cos_similarity        |      0.0003      |          0.0004          |      4       |     0.0012     |
|           cos_similarity          |      0.0165      |          0.0214          |      4       |     0.0658     |
|      get_max_xy_from_cos_map      |      0.0002      |          0.0003          |      4       |     0.0009     |
| get_frame_to_frame_correspondence |      1.1630      |          1.1630

# Test cos similarity speed